In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from datetime import date, datetime

In [2]:
atividades = pd.read_csv(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Atividades.csv", sep=';', encoding='utf-8')
print(atividades)

           Evento   Extenso  Contagem  Usuarios  dia  mês   ano        Data
0       page_view  20230828      1967        94   28    8  2023  28/08/2023
1       page_view  20231025      1922       180   25   10  2023  25/10/2023
2       page_view  20230807      1903       108    7    8  2023  07/08/2023
3       page_view  20240121      1884       187   21    1  2024  21/01/2024
4       page_view  20231005      1860       164    5   10  2023  05/10/2023
...           ...       ...       ...       ...  ...  ...   ...         ...
2034        click  20240413        48         6   13    4  2024  13/04/2024
2035  first_visit  20240411        40        40   11    4  2024  11/04/2024
2036  first_visit  20240408        38        38    8    4  2024  08/04/2024
2037        click  20240409        30         6    9    4  2024  09/04/2024
2038        click  20240411        25        14   11    4  2024  11/04/2024

[2039 rows x 8 columns]


In [3]:
data_inicio = '02-08-2023'
data_inicio_titulo = data_inicio
data_inicio = datetime.strptime(data_inicio, '%d-%m-%Y').date()
data_fim = '16-04-2024'
data_fim_titulo = data_fim
data_fim = datetime.strptime(data_fim, '%d-%m-%Y').date()
atividades['data_corte'] = ''
for ind in range(0, len(atividades)):
    atividades['data_corte'][ind] = datetime.strptime(atividades['Data'][ind], '%d/%m/%Y').date()
atividades = atividades[atividades['data_corte'] >= data_inicio]
atividades = atividades[atividades['data_corte'] <= data_fim]

In [33]:
buscas = ['click', 'first_visit', 'page_view', 'user_engagement', 'form_start', 'form_submit', 'scroll', 'session start']
buscas = sorted(buscas)
rodada = 0
colunas = ['Evento', 'Extenso', 'Contagem', 'Usuarios', 'dia', 'mes', 'ano', 'Data']
pag_buscaf = pd.DataFrame(columns=colunas)
for ind in range(0, len(buscas)):
    nome = buscas[ind]
    atividades['nome_busca'] = atividades.Evento.str.contains(nome)
    pag_busca1 = atividades.query("nome_busca == True")
    pag_busca1['nome'] = buscas[ind]
    rodada = rodada + 1
    if rodada == 0:
        pag_buscaf = pag_busca1
    else:
        pag_buscaf = pd.concat([pag_buscaf, pag_busca1])
    atividades['nome_busca'] = False
pag_buscaf = pag_buscaf.drop(['Evento', 'Usuarios', 'dia', 'mês', 'ano', 'nome_busca'], axis=1)
print(pag_buscaf)

     Contagem        Data   Extenso  data_corte  mes             nome
604       359  24/09/2023  20230924  2023-09-24  NaN            click
648       281  25/10/2023  20231025  2023-10-25  NaN            click
649       280  08/08/2023  20230808  2023-08-08  NaN            click
692       236  25/08/2023  20230825  2023-08-25  NaN            click
695       233  21/09/2023  20230921  2023-09-21  NaN            click
...       ...         ...       ...         ...  ...              ...
1990      770  13/04/2024  20240413  2024-04-13  NaN  user_engagement
1991      756  11/04/2024  20240411  2024-04-11  NaN  user_engagement
1995      720  09/04/2024  20240409  2024-04-09  NaN  user_engagement
1997      595  14/04/2024  20240414  2024-04-14  NaN  user_engagement
1998      581  08/04/2024  20240408  2024-04-08  NaN  user_engagement

[1784 rows x 6 columns]


In [34]:
pag_buscaf = pag_buscaf.groupby(['Extenso', 'data_corte', 'nome'], as_index=False).sum()
pag_buscaf = pag_buscaf.sort_values(by=['Extenso', 'data_corte', 'nome'], ascending=True)
pag_buscaf = pag_buscaf.reset_index(drop=True)
print(pag_buscaf)

       Extenso  data_corte             nome  Contagem        Data  mes
0     20230804  2023-08-04            click        74  04/08/2023    0
1     20230804  2023-08-04      first_visit        69  04/08/2023    0
2     20230804  2023-08-04       form_start       116  04/08/2023    0
3     20230804  2023-08-04      form_submit       211  04/08/2023    0
4     20230804  2023-08-04        page_view      1049  04/08/2023    0
...        ...         ...              ...       ...         ...  ...
1779  20240414  2024-04-14       form_start        80  14/04/2024    0
1780  20240414  2024-04-14      form_submit       236  14/04/2024    0
1781  20240414  2024-04-14        page_view       738  14/04/2024    0
1782  20240414  2024-04-14           scroll       298  14/04/2024    0
1783  20240414  2024-04-14  user_engagement       595  14/04/2024    0

[1784 rows x 6 columns]


In [38]:
fig = px.line(pag_buscaf, x="data_corte", y="Contagem", color='nome', title='Número médio de clicks até Abril 2024', markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.add_hline(y=59, line_width=3, line_dash="dash", line_color="red")
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Atividades_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [7]:
count_mes = atividades.groupby(['mês', 'ano'], as_index=False).sum()
count_mes = count_mes.sort_values(by=['ano', 'mês'], ascending=True)
count_mes['mesano'] = ''
for ind in range(0, len(count_mes)):
    count_mes['mesano'][ind] = str(count_mes['ano'][ind]) + str(count_mes['mês'][ind])
print(count_mes)

   mês   ano     Extenso  Contagem  Usuarios   dia  nome_busca  mesano
4    8  2023  4531703120     95798     15902  3920       False   20238
5    9  2023  4855419720     97672     16002  3720       False   20239
6   10  2023  5017291968     83382     16238  3968       False  202310
7   11  2023  4855467720     92649     17989  3720       False  202311
8   12  2023  5017341568     81027     17217  3968       False  202312
0    1  2024  4999308640     89965     15008  3940       False   20241
1    2  2024  4695729880     71335     15373  3480       False   20242
2    3  2024  5019598368     77961     13936  3968       False   20243
3    4  2024  2266925640     35314      6488   840       False   20244


In [8]:
fig = px.line(count_mes, x="mesano", y="Contagem", title='Atividades por Mês de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Atividades_Mes" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [9]:
qtd_antes = 0
qtd_apos = 0
mes_antes = 0
mes_apos = 0
for ind in range(0, len(count_mes)):
    if count_mes['ano'][ind] == 2024:
        if count_mes['mês'][ind] == 3:
            print('Entrou')
        else:
            if count_mes['mês'][ind] == 1:
                qtd_antes = qtd_antes + int(count_mes['Contagem'][ind])
                mes_antes = mes_antes + 1
            else:    
                qtd_apos = qtd_apos + int(count_mes['Contagem'][ind])
                mes_apos = mes_apos + 1
    else:
        qtd_antes = qtd_antes + int(count_mes['Contagem'][ind])
        mes_antes = mes_antes + 1
qtd_apos = round((qtd_apos / mes_apos),0)
qtd_antes = round((qtd_antes / mes_antes),0)
aumento = round((((qtd_apos - qtd_antes) / qtd_antes) * 100),2)
print(aumento)

Entrou
-40.81


In [27]:
ativi_atrib = pd.read_csv(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Atividade_atributo.csv", sep=';', encoding='utf-8')
fig = px.line(ativi_atrib, x="Data", y="Submit", color='Pagina', title='Consultas por dia de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.add_hline(y=5.38, line_width=3, line_dash="dash", line_color="green")
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Ativatrib_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [11]:
ativi_atrib['saldo'] = ativi_atrib['Submit'] - ativi_atrib['Acessos']
fig = px.line(ativi_atrib, x="Data", y="saldo", color='Pagina', title='Saldo de submit por dia de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Saldosub_" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [12]:
print(ativi_atrib)
count_mes2 = ativi_atrib.groupby(['Mes', 'Ano'], as_index=False).sum()
count_mes2 = count_mes2.sort_values(by=['Ano', 'Mes'], ascending=True)
count_mes2['mesano'] = ''
for ind in range(0, len(count_mes2)):
    count_mes2['mesano'][ind] = str(count_mes2['Ano'][ind]) + str(count_mes2['Mes'][ind])
print(count_mes2)

           Pagina   Extenso  Submit  Acessos        Data  Mes   Ano  saldo
0     /adversario  20230803      21       27  03/08/2023    8  2023     -6
1     /adversario  20230804      28       29  04/08/2023    8  2023     -1
2     /adversario  20230805      17       17  05/08/2023    8  2023      0
3     /adversario  20230806      22       24  06/08/2023    8  2023     -2
4     /adversario  20230807      36       41  07/08/2023    8  2023     -5
...           ...       ...     ...      ...         ...  ...   ...    ...
1449     /tecnico  20240410      28        7  10/04/2024    4  2024     21
1450     /tecnico  20240411      11        3  11/04/2024    4  2024      8
1451     /tecnico  20240412      20        5  12/04/2024    4  2024     15
1452     /tecnico  20240413       6        2  13/04/2024    4  2024      4
1453     /tecnico  20240414      24        6  14/04/2024    4  2024     18

[1454 rows x 8 columns]
   Mes   Ano     Extenso  Submit  Acessos  saldo  mesano
4    8  2023  3398

In [13]:
qtd_antes = 0
qtd_apos = 0
mes_antes = 0
mes_apos = 0
for ind in range(0, len(count_mes)):
    if count_mes2['Ano'][ind] == 2024:
        if count_mes2['Mes'][ind] < 3:
            qtd_antes = qtd_antes + int(count_mes2['saldo'][ind])
            mes_antes = mes_antes + 1
        else:    
            qtd_apos = qtd_apos + int(count_mes2['saldo'][ind])
            mes_apos = mes_apos + 1
    else:
        qtd_antes = qtd_antes + int(count_mes2['saldo'][ind])
        mes_antes = mes_antes + 1
qtd_apos = round((qtd_apos / mes_apos),0)
qtd_antes = round((qtd_antes / mes_antes),0)
aumento = round((((qtd_apos - qtd_antes) / qtd_antes) * 100),2)
print(qtd_apos)
print(qtd_antes)
print(aumento)

2400.0
-515.0
-566.02


In [14]:
fig = px.line(count_mes2, x="mesano", y="saldo", title='Desistencia por Mês de ' + str(data_inicio_titulo) + ' até ' + str(data_fim_titulo), markers=True)
fig.update_layout(xaxis_tickangle=-45)
fig.write_html(r"C:\Users\rodli\OneDrive\Documentos\Mestrado\Arquivos\Graficos\Desistencia_Mes" + str(data_inicio_titulo) + "_" + str(data_fim_titulo) + ".html")
fig.show()

In [24]:
atributos_arb = ativi_atrib[ativi_atrib['Pagina'] == '/arbitro']
qtd_submit = atributos_arb['Submit'].sum()
qtd_dias = atributos_arb['Submit'].count()
qtd_submit_antes = atributos_arb[atributos_arb['Extenso'] < 20240301]['Submit'].sum()
qtd_submit_depois = atributos_arb[atributos_arb['Extenso'] > 20240228]['Submit'].sum()
qtd_dias_antes = atributos_arb[atributos_arb['Extenso'] < 20240301]['Submit'].count()
qtd_dias_depois = atributos_arb[atributos_arb['Extenso'] > 20240228]['Submit'].count()
media = round((qtd_submit / qtd_dias),2)
media_antes = round((qtd_submit_antes / qtd_dias_antes),2)
media_depois = round((qtd_submit_depois / qtd_dias_depois),2)
print(atributos_arb)
print(qtd_submit)
print(qtd_dias)
print(qtd_submit_antes)
print(qtd_dias_antes)
print(qtd_submit_depois)
print(qtd_dias_depois)
print(media)
print(media_antes)
print(media_depois)

        Pagina   Extenso  Submit  Acessos        Data  Mes   Ano
235   /arbitro  20230803      11       11  03/08/2023    8  2023
236   /arbitro  20230804       4        6  04/08/2023    8  2023
237   /arbitro  20230805      10       13  05/08/2023    8  2023
238   /arbitro  20230806       2        3  06/08/2023    8  2023
239   /arbitro  20230807       5        6  07/08/2023    8  2023
...        ...       ...     ...      ...         ...  ...   ...
1423  /arbitro  20240410      35       10  10/04/2024    4  2024
1424  /arbitro  20240411      20        8  11/04/2024    4  2024
1425  /arbitro  20240412       5        1  12/04/2024    4  2024
1426  /arbitro  20240413      32        8  13/04/2024    4  2024
1427  /arbitro  20240414      17        4  14/04/2024    4  2024

[226 rows x 7 columns]
1215
226
825
188
400
39
5.38
4.39
10.26


In [29]:
buscas = ['click']
buscas = sorted(buscas)
rodada = 0
colunas = ['Evento', 'Extenso', 'Contagem', 'Usuarios', 'dia', 'mes', 'ano', 'Data']
pag_buscaf = pd.DataFrame(columns=colunas)
for ind in range(0, len(buscas)):
    nome = buscas[ind]
    atividades['nome_busca'] = atividades.Evento.str.contains(nome)
    pag_busca1 = atividades.query("nome_busca == True")
    pag_busca1['nome'] = buscas[ind]
    rodada = rodada + 1
    if rodada == 0:
        pag_buscaf = pag_busca1
    else:
        pag_buscaf = pd.concat([pag_buscaf, pag_busca1])
    atividades['nome_busca'] = False
pag_buscaf = pag_buscaf.drop(['Evento', 'Usuarios', 'dia', 'mês', 'ano', 'nome_busca'], axis=1)
print(pag_buscaf)

     Contagem        Data   Extenso  data_corte  mes   nome
604       359  24/09/2023  20230924  2023-09-24  NaN  click
648       281  25/10/2023  20231025  2023-10-25  NaN  click
649       280  08/08/2023  20230808  2023-08-08  NaN  click
692       236  25/08/2023  20230825  2023-08-25  NaN  click
695       233  21/09/2023  20230921  2023-09-21  NaN  click
...       ...         ...       ...         ...  ...    ...
2028       83  12/04/2024  20240412  2024-04-12  NaN  click
2031       66  08/04/2024  20240408  2024-04-08  NaN  click
2034       48  13/04/2024  20240413  2024-04-13  NaN  click
2037       30  09/04/2024  20240409  2024-04-09  NaN  click
2038       25  11/04/2024  20240411  2024-04-11  NaN  click

[254 rows x 6 columns]


In [32]:
qtd_ativ_dias = pag_buscaf['Contagem'].count()
click_qtd = pag_buscaf['Contagem'].sum()
media = round((click_qtd / qtd_ativ_dias),2)
print(media)

58.99
